## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-14-15-51-39 +0000,bbc,Nigeria declares public holiday to honour form...,https://www.bbc.com/news/articles/c93kxl08glwo
1,2025-07-14-15-43-00 +0000,bbc,Tata promises electric arc by 2027 as work begins,https://www.bbc.com/news/articles/crl0176kz6po
2,2025-07-14-15-25-58 +0000,bbc,UK to start processing Syrian asylum claims again,https://www.bbc.com/news/articles/cy4y2e8neywo
3,2025-07-14-15-15-00 +0000,wsj,EU Trade Chief to Contact U.S. Officials After...,https://www.wsj.com/economy/trade/eu-trade-chi...
4,2025-07-14-15-13-20 +0000,nypost,"Wildfire destroys historic Grand Canyon lodge,...",https://nypost.com/2025/07/14/us-news/wildfire...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2344/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
21,trade,5
132,found,3
69,trump,3
218,tariffs,3
76,war,3


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
10,2025-07-14-14-48-00 +0000,wsj,Trump is threatening to ramp up the U.S. trade...,https://www.wsj.com/economy/trade/european-uni...,32
44,2025-07-14-02-00-00 +0000,wsj,Chinese manufacturers looking to dodge steep U...,https://www.wsj.com/economy/trade/vietnam-chin...,32
32,2025-07-14-06-08-00 +0000,wsj,"China’s exports grew at a faster clip in June,...",https://www.wsj.com/economy/trade/chinas-expor...,22
17,2025-07-14-11-54-12 +0000,nypost,"At least 9 dead, 30 hospitalized after fire te...",https://nypost.com/2025/07/14/us-news/9-dead-3...,19
11,2025-07-14-14-26-38 +0000,nypost,Student accused of killing newborn and throwin...,https://nypost.com/2025/07/14/us-news/student-...,18


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
10,32,2025-07-14-14-48-00 +0000,wsj,Trump is threatening to ramp up the U.S. trade...,https://www.wsj.com/economy/trade/european-uni...
17,19,2025-07-14-11-54-12 +0000,nypost,"At least 9 dead, 30 hospitalized after fire te...",https://nypost.com/2025/07/14/us-news/9-dead-3...
11,18,2025-07-14-14-26-38 +0000,nypost,Student accused of killing newborn and throwin...,https://nypost.com/2025/07/14/us-news/student-...
33,18,2025-07-14-05-26-09 +0000,nypost,Biden admits he didn’t approve each individual...,https://nypost.com/2025/07/14/us-news/biden-de...
44,18,2025-07-14-02-00-00 +0000,wsj,Chinese manufacturers looking to dodge steep U...,https://www.wsj.com/economy/trade/vietnam-chin...
32,17,2025-07-14-06-08-00 +0000,wsj,"China’s exports grew at a faster clip in June,...",https://www.wsj.com/economy/trade/chinas-expor...
43,16,2025-07-14-02-00-16 +0000,nypost,West Virginia couple pulled over after cops sp...,https://nypost.com/2025/07/13/us-news/west-vir...
14,14,2025-07-14-13-30-19 +0000,nypost,High-speed train linking NYC to Los Angeles pr...,https://nypost.com/2025/07/14/us-news/la-to-ny...
26,14,2025-07-14-09-30-00 +0000,wsj,Food banks across the country were already str...,https://www.wsj.com/economy/food-banks-are-run...
22,13,2025-07-14-10-07-58 +0000,nypost,"Gunman in Kentucky church shooting declared, ‘...",https://nypost.com/2025/07/14/us-news/gunman-i...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
